In [7]:
import os
import struct
import numpy as np
from array import array as pyarray
import torch

def load_emnist(dataset="training", letters=np.arange(27), path="data/EMNIST/raw/", size = 6000):
    if dataset == "training":
        fname_img = os.path.join(path, 'emnist-letters-train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'emnist-letters-train-labels-idx1-ubyte')
    elif dataset == "testing":
        fname_img = os.path.join(path, 'emnist-letters-test-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'emnist-letters-test-labels-idx1-ubyte')

    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    flbl = open(fname_lbl, 'rb')
    magic_nr, size = struct.unpack(">II", flbl.read(8))
    lbl = pyarray("b", flbl.read())
    lbl = np.array(lbl).astype(int)

    flbl.close()

    fimg = open(fname_img, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", fimg.read(16))
    img = pyarray("B", fimg.read())
    fimg.close()

    ind = [ k for k in range(size) if lbl[k] in letters ]
    N = len(ind)
    images = np.zeros((N, rows, cols))
    labels = np.zeros((N, 1))
    for i in range(N): #int(len(ind) * size/100.)):
        tmp1 = (img[ ind[i]*rows*cols : (ind[i]+1)*rows*cols ])
        tmp = np.array(tmp1).reshape((rows, cols))
        images[i] = tmp
        labels[i] = lbl[ind[i]] - 1
    labels = np.array([label[0] for label in labels])
    # labels = labels.reshape(labels.shape[0], 1)
    images = images.reshape(images.shape[0], images.shape[1], images.shape[2], 1)
    return images, labels

X_train, y_train = load_emnist()
X_test, y_test = load_emnist(dataset='testing')

In [23]:
from torchvision import datasets, transforms
from torch.utils import data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

BATCH_SIZE=32
tensor_x = torch.LongTensor(X_train)
tensor_y = torch.LongTensor(y_train)
train_dataset = data.TensorDataset(tensor_x,tensor_y)
trainloader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
tensor_x = torch.LongTensor(X_test)
tensor_y = torch.LongTensor(y_test)
test_dataset = data.TensorDataset(tensor_x,tensor_y)
testloader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [11]:
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data
from time import time
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=2)
        self.pool1 = nn.MaxPool2d(2,2)
        self.pool2 = nn.MaxPool2d(2,2)
        self.linear2 = nn.Linear(6 * 6 * 16, 10)

    def forward(self,X):
        X = self.pool1(F.relu(self.conv1(X)))
        X = self.pool2(F.relu(self.conv2(X)))
        X = X.view(-1, 6 * 6 * 16)
        X = self.linear2(X)
        return X

model = Net()
print(model)

Net(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(2, 2), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear2): Linear(in_features=576, out_features=10, bias=True)
)


In [24]:
device = torch.device("cpu")
criterion = nn.CrossEntropyLoss()
images, labels = next(iter(trainloader))
print(images.permute(0,3,1,2).shape)
images = images.permute(0,3,1,2)
model(images)
logps = model(images.to(device)) #log probabilities
loss = criterion(logps, labels.to(device)) #calculate t

torch.Size([32, 1, 28, 28])


RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'weight' in call to _thnn_conv2d_forward

In [12]:
def accuracy(testloader, epoch):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs= model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print("Accuracy", 100 * correct / total)
        return 100 * correct / total

# accuracy(valloader, 0)

In [13]:
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
time0 = time()
epochs = 2
for e in range(epochs):
    running_loss = 0
    running_val_loss = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        output = model(images)
        loss = criterion(output, labels)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        output = model(images)
        loss = criterion(output, labels)
        running_val_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
        print("Epoch {} - Validation loss: {}".format(e, running_val_loss/len(valloader)))
        print("on train set:")
        accuracy(trainloader, e)
        print("on test set:")
        accuracy(testloader, e)
print("\nTraining Time (in minutes) =",(time()-time0)/60)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight 8 1 3 3, but got 3-dimensional input of size [32, 28, 28] instead